In [1]:
import statsmodels.formula.api as smf

def bidirectional_elimination(data, response):
    """Linear model designed by bidirectional elimination.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
    with an intercept
    selected by bidirectional elimination
    evaluated by adjusted R-squared
    entrance tolerance is 0.05
    exit tolerance is 0.10

    References:
    -----------
    [1] https://planspace.org/20150423-forward_selection_with_statsmodels/\n
    [2] https://en.wikipedia.org/wiki/Stepwise_regression
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score = 0.0
    while remaining:
        good_scores_with_candidates = []
        for candidate in remaining:
            formula = "{0} ~ {1} + 1".format(response,
                                             ' + '.join(selected + [candidate]))
            model = smf.ols(formula, data).fit()
            score = model.rsquared_adj
            if score > current_score and model.pvalues[str(candidate)] < 0.05:
                good_scores_with_candidates.append((score, candidate))
        if good_scores_with_candidates == []:
            formula = "{0} ~ {1} + 1".format(response, ' + '.join(selected))
            model = smf.ols(formula, data).fit()
            break
        good_scores_with_candidates.sort()
        best_candidate = good_scores_with_candidates.pop()[1]
        selected.append(best_candidate)
        remaining.remove(best_candidate)
        formula = "{0} ~ {1} + 1".format(response, ' + '.join(selected))
        model = smf.ols(formula, data).fit()
        for x in model.params.index[1:]:
            if model.pvalues[x] > 0.10:
                selected.remove(x)
                remaining.add(x)
        current_score = score
    return model

In [2]:
import pandas as pd

data = pd.read_csv('C:\\Users\\poposoto\\Desktop\\model\\810\\股票变化.csv', index_col=0)
model = bidirectional_elimination(data, 'y')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     193.9
Date:                Sat, 10 Aug 2019   Prob (F-statistic):           8.40e-10
Time:                        17:34:50   Log-Likelihood:                -91.561
No. Observations:                  15   AIC:                             191.1
Df Residuals:                      11   BIC:                             194.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     75.8067     71.109      1.066      0.3

D:\anaconda\lib\site-packages\scipy\stats\stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))
